# Setup

In [1]:
import functools

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

import dataclasses as dc
import datetime
import decimal
import typing

import dbcontroller as dbc

# URL
DATABASE_URL = "sqlite:///example.db"

# Base
Base = declarative_base()

# Manager
SQL = functools.partial(dbc.SQL, DATABASE_URL)

# Model
model = dbc.Model(sql=Base)


# DateTime Functions
class Date:
    datetime = lambda: datetime.datetime.now()
    date = lambda: datetime.date.today()
    time = lambda: datetime.datetime.now().time()

# Models

In [ ]:
# Create your <types> here.
@model.sql
class Product:
    name: str
    aliases: list[str] | None = None
    stock: int | None = None
    is_available: bool | None = None
    created_on: datetime.datetime = dc.field(default_factory=Date.datetime)
    available_from: datetime.date = dc.field(default_factory=Date.date)
    same_day_shipping_before: datetime.time = dc.field(default_factory=Date.time)
    price: decimal.Decimal | None = None
    notes: list[dbc.Text] = dc.field(default_factory=list)
    is_object: dbc.JSON = dc.field(default_factory=dict)

    async def category(self) -> "Category":
        return Category(name="awesome")


@model.sql
class Category:
    name: str

# Test

In [2]:
# Regular Engine
engine = create_engine(DATABASE_URL, echo=True)

# Register Tables & Load -> Lazy-Loaded Tables (aka: Types)
dbc.Admin.register([Product, Category])
dbc.Admin.load()

# Create Tables
Base.metadata.create_all(engine)